In [5]:
!pip install deep-forest

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 32.1 MB/s eta 0:00:00


In [6]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
import pandas as pd
from time import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder
from sklearn.metrics import accuracy_score

from deepforest import CascadeForestClassifier

from yellowbrick.classifier import ROCAUC

import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive/')

# Insert the directory
import sys
sys.path.insert(0,'/content/drive/My Drive/')

Mounted at /content/drive/


In [7]:
# load data
df=pd.read_csv('/content/drive/MyDrive/data/UNSW_2018_IoT_Botnet_Final_10_Best.csv',sep=";")  
del df['Unnamed: 0']

#df=pd.read_csv('/content/drive/MyDrive/data/300.csv',sep=",") 


df = df.sample(frac=1)
print("shape of data: ",df.shape)
print("----------------------------------")

df.head()

shape of data:  (3668522, 19)
----------------------------------


,pkSeqID,proto,saddr,sport,daddr,dport,seq,stddev,N_IN_Conn_P_SrcIP,min,state_number,mean,N_IN_Conn_P_DstIP,drate,srate,max,attack,category,subcategory
3433037,3433038,udp,192.168.100.147,56390,192.168.100.3,80,17957,0.830299,87,2.936662,4,3.976313,100,0.0,1.007614,4.968832,1,DDoS,UDP
2876379,2876380,udp,192.168.100.149,61207,192.168.100.3,80,247763,1.180276,100,2.027345,4,3.207621,100,0.0,0.473772,4.387897,1,DDoS,UDP
738051,738052,udp,192.168.100.150,30242,192.168.100.3,80,120775,0.123128,100,4.448598,4,4.615377,100,0.0,0.390466,4.746838,1,DoS,UDP
3088318,3088319,udp,192.168.100.150,49244,192.168.100.3,80,197541,1.901020,44,0.000000,4,2.688448,100,0.0,0.494538,4.033067,1,DDoS,UDP
824654,824655,udp,192.168.100.150,59205,192.168.100.3,80,207378,0.439272,100,3.596719,4,4.339324,100,0.0,0.415569,4.730598,1,DoS,UDP


# Exploratory Data Analysis

# Extracting 10 best features

In [8]:
ten_best_features = df[['seq','stddev','N_IN_Conn_P_SrcIP', 'min', 'state_number', 'mean', 'N_IN_Conn_P_DstIP',
       'drate', 'srate', 'max','category']]
#target_features = df[['attack','category','subcategory']]

# Data Preprocessing


In [327]:
# Label Encoding the target columns

le = LabelEncoder()
ten_best_features['category'] = le.fit_transform(ten_best_features['category'])
#target_features['subcategory'] = le.fit_transform(target_features['subcategory'])

#target_features.head()

classes = list(ten_best_features['category'].unique()) 
classes

[3, 1, 2, 0]

In [9]:
ten_best_features = ten_best_features.values

y = ten_best_features[:,10]
X = ten_best_features[:,0:10]

# Train-test Split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)


# Scaling the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [329]:
# number of class
print('number classes :', len(np.unique(y_train)))

number classes : 4


# Extratrees Deep

In [330]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("/content/drive/My Drive/examples/category.json")
gc = GCForest(config)

In [331]:
# X_enc is the concatenated predict_proba result of DeepExtraTrees
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

INFO:gcforest.cascade.cascade_classifier:X_groups_train.shape=[(210, 10)],y_train.shape=(210,),X_groups_test.shape=[(90, 10)],y_test.shape=(90,)
INFO:gcforest.cascade.cascade_classifier:group_dims=[10]
INFO:gcforest.cascade.cascade_classifier:group_starts=[0]
INFO:gcforest.cascade.cascade_classifier:group_ends=[10]
INFO:gcforest.cascade.cascade_classifier:X_train.shape=(210, 10),X_test.shape=(90, 10)
INFO:gcforest.cascade.cascade_classifier:[layer=0] look_indexs=[0], X_cur_train.shape=(210, 10), X_cur_test.shape=(90, 10)
INFO:gcforest.estimators.kfold_wrapper:Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=90.48%
INFO:gcforest.estimators.kfold_wrapper:Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=80.95%
INFO:gcforest.estimators.kfold_wrapper:Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=85.71%
INFO:gcforest.estimators.kfold_wrapper:Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=95.24%
INFO:gcforest.estimators.kfold_wrapper:Accuracy(layer

In [332]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of DeepExtratTrees = {:.6f} %".format(acc * 100))
 # Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

INFO:gcforest.cascade.cascade_classifier:X_groups_test.shape=[(90, 10)]
INFO:gcforest.cascade.cascade_classifier:group_dims=[10]
INFO:gcforest.cascade.cascade_classifier:X_test.shape=(90, 10)
INFO:gcforest.cascade.cascade_classifier:[layer=0] look_indexs=[0], X_cur_test.shape=(90, 10)


Test Accuracy of DeepExtratTrees = 94.444444 %
[[24  0  1  0]
 [ 1 23  1  0]
 [ 1  0 18  0]
 [ 1  0  0 20]]


              precision    recall  f1-score   support

         0.0       0.89      0.96      0.92        25
         1.0       1.00      0.92      0.96        25
         2.0       0.90      0.95      0.92        19
         3.0       1.00      0.95      0.98        21

    accuracy                           0.94        90
   macro avg       0.95      0.94      0.95        90
weighted avg       0.95      0.94      0.95        90



In [ ]:
from deepforest import CascadeForestClassifier

det = CascadeForestClassifier(random_state=1)
det.fit(X_train, y_train)
y_pred = det.predict(X_test)
acc = accuracy_score(y_test, y_pred) * 100
print("\nTesting Accuracy: {:.3f} %".format(acc))

[2023-01-11 09:36:02.299] Start to fit the model:
[2023-01-11 09:36:02.301] Fitting cascade layer = 0 


In [ ]:
# predict
y_pred = det.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of CascadeForestClassifier = {:.6f} %".format(acc * 100))
 # Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

In [ ]:
    # Load multi-class classification dataset
    
    det = CascadeForestClassifier()

    # Encode the non-numeric columns
    X = OrdinalEncoder().fit_transform(X)
    y = LabelEncoder().fit_transform(y)

    # Create the train and test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    # Instaniate the classification model and visualizer
   
    visualizer = ROCAUC(det, classes=['DoS', 'DoSS', 'DoSSS', 'SDoS'])

    visualizer.fit(X_train, y_train)        # Fit the training data to the visualizer
    visualizer.score(X_test, y_test)        # Evaluate the model on the test data
    visualizer.show()                       # Finalize and render the figure